In [1]:
from spotify.spotify_service import SpotifyManager
import os

def choose_playlist(spotify_manager):
    playlists = spotify_manager.get_user_playlists()
    for i, playlist in enumerate(playlists['items']):
        print(f"{i+1}: {playlist['name']}")

    # choice = int(input("Enter the number of the playlist you want to analyze: "))
    choice=26
    return playlists['items'][choice - 1]['id']

def main():
    spotify_manager = SpotifyManager()
    playlist_id = choose_playlist(spotify_manager)
    full_track_data = spotify_manager.get_full_track_data(playlist_id)

    for track_data in full_track_data:
        track = track_data['track']
        print(f"{track['name']} by {track['artists'][0]['name']}")
        audio_features = track.get('audio_features')
        if audio_features:
            print(f"  - Tempo: {audio_features['tempo']}")
            print(f"  - Key: {audio_features['key']}")
            # Add more features as needed
    print(track_data)

In [2]:
spotify_manager = SpotifyManager()
playlist_id = choose_playlist(spotify_manager)
full_track_data = spotify_manager.get_full_track_data(playlist_id)

1: Facemelts Top 50 Songs of 23 (Official)
2: Bass
3: House Party
4: Home & Hosed
5: Something More
6: triple j's 40 Years of Music
7: Soft Office
8: Facemelts February 
9: ELETRONICAS 2024 ⚡ MAIS TOCADAS
10: Playlist #112 
11: Your Top Songs 2023
12: This Is Clutch
13: Paul Mcartney 2023
14: Bem
15: This Is Slightly Stoopid
16: Indie Halloween
17: Halloween 2023
18: Halloween Playlist 2024🎃
19: playlist #184
20: Funk Mix
21: Instrumental Rock
22: Garage band
23: This Is Pitty
24: This Is Bruno Mars
25: Foo fighters setlist 2023
26: Drums
27: Rock Titans
28: kilby girl energy
29: Funky Psychedelic Grooves
30: Parcels Discography
31: French
32: Foo Fighters 2023
33: Funky
34: New Be
35: Voice exercises 
36: Canto
37: 50s Party
38: New rock
39: GREATEST DRUMMERS OF ALL TIME
40: Ale cantar
41: Indie Rock Gems 2
42: Pagode Mix
43: Calm Indie Rock
44: Calm Alternative Rock
45: Calming Piano Instrumentals
46: 2000s Happy Road Trip
47: Hidden Gems of Rock
48: Indie Rock Hidden Gems
49: Noivad

In [9]:
full_track_data[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri', 'audio_features'])

In [11]:
full_track_data[0]['track']['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/3ZGr7nQBXDU2WhyXgRVbt0'},
  'href': 'https://api.spotify.com/v1/artists/3ZGr7nQBXDU2WhyXgRVbt0',
  'id': '3ZGr7nQBXDU2WhyXgRVbt0',
  'name': 'Sticky Fingers',
  'type': 'artist',
  'uri': 'spotify:artist:3ZGr7nQBXDU2WhyXgRVbt0'}]